# **Trabalho 3**

In [ ]:
!pip install ktrain

In [81]:
import pandas as pd
import requests,json,ktrain,tweepy,io
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from ktrain import text

In [ ]:
log=pd.read_csv('keys.csv')
consumerkey=log['key'][0]
consumersecret=log['key'][1]
accesstoken=log['key'][2]
accesstokensecret=log['key'][3]

In [ ]:
client = tweepy.Client(accesstoken)

#Lendo twetts dos filmes thor amor e trovao, Sonic e Elvis.
res1 =  tweepy.Paginator(client.search_recent_tweets, query="thorloveandthunder", tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=200)
res2 =  tweepy.Paginator(client.search_recent_tweets, query="Sonic", tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=200)
res3 =  tweepy.Paginator(client.search_recent_tweets, query="elvis", tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=200)

In [ ]:
def remove_caracteres(text,carc):
    for i in range(len(carc)):
            text = text.replace(carc[i],"")
    return text

def remove_stop_words(text,stops):
    for i in stops:
            idx = text.find(i)
            if(idx != -1):
                text = text[0:idx]+text[(idx+len(i)):len(text)]
    return text

def load_tweets(size=1000):
    tweets_thor = []
    tweets_sonic = []
    tweets_elvis = []
    for i in range(10):
        res1 = tweepy.Paginator(client.search_recent_tweets, query="thorloveandthunder", tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=200)
        res2 = tweepy.Paginator(client.search_recent_tweets, query="Sonic", tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=200)
        res3 = tweepy.Paginator(client.search_recent_tweets, query="elvis", tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=200)
        tweets_thor.append(res1)
        tweets_sonic.append(res2)
        tweets_elvis.append(res3)
    return tweets_thor, tweets_sonic, tweets_elvis


In [ ]:
res1,res2,res3 = load_tweets()


#Guardando os twetts em im arquivo json
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)
stopwords.update(["RT","thor","thunder","ThorLoveAndThunder","https","\n"])
caracteres = "#@:;|_{}[]()"
info = {"thor":{"text":[]},"Sonic":{"text":[]},"elvis":{"text":[]}}
with open('movies.json', 'w') as outfile:
    for j in range(10):
        for response in res1[j]:
            response.text = remove_caracteres(response.text,caracteres)
            response.text = remove_stop_words(response.text,stopwords)
            info["thor"]["text"].append(response.text)
        for response2 in res2[j]:
            response2.text = remove_caracteres(response2.text,caracteres)
            response2.text = remove_stop_words(response2.text,stopwords)
            info["Sonic"]["text"].append(response2.text)
        for response3 in res3[j]:
            response3.text = remove_caracteres(response3.text,caracteres)
            response3.text = remove_stop_words(response3.text,stopwords)
            info["elvis"]["text"].append(response3.text)
    outfile.write(json.dumps(info))

In [5]:
s = requests.get('https://raw.githubusercontent.com/ragero/text-collections/master/complete_texts_csvs/review_polarity.csv').content
dataset = pd.read_csv(io.StringIO(s.decode('utf-8')),na_values=['?'],skipinitialspace = True)
dataset = dataset.sample(frac=1)

In [ ]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(dataset, 
                                                                   'text',
                                                                   label_columns='class',
                                                                   maxlen=64, 
                                                                   max_features=50000,
                                                                   preprocess_mode='bert',
                                                                   lang=None,
                                                                   val_pct = 0.1,
                                                                   )

In [7]:
model = text.text_classifier('bert', (X_train, y_train) , preproc=preprocess)
classifier = ktrain.get_learner(model, 
                             train_data=(X_train, y_train), 
                             val_data=(X_test, y_test),
                             batch_size=64
                             )

Is Multi-Label? False
maxlen is 64
done.


In [21]:
classifier.fit_onecycle(0.00002,8)
classifier.validate()
predictor = ktrain.get_predictor(classifier.model, preprocess)
predictor.predict('this is a good movie')

              precision    recall  f1-score   support

           0       0.68      0.63      0.65       102
           1       0.64      0.69      0.67        98

    accuracy                           0.66       200
   macro avg       0.66      0.66      0.66       200
weighted avg       0.66      0.66      0.66       200



array([[64, 38],
       [30, 68]])